<a href="https://colab.research.google.com/github/stefanoridolfi/Coding_ML/blob/master/2_level__con_Bias_dataset_quadrato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:



commento="VERSIONE con BIAS"
import matplotlib.pyplot as plt
import random
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

import pandas as pd
from csv import reader
from math import sqrt,exp
import matplotlib.pyplot as plt




@np.vectorize

def posizione(classe):
  
  if classe==1:
    formato = 'ro'
  else:
    formato = 'go'
  return formato

def posizione_neurale(p):
    if p>0.5:
      formato = 'ro'
      out=1
    else:
      formato = 'go'
      out=0
    return out,formato



def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid


def float_column_to_int(dataset, column):
  for row in dataset:
    row[column] = int(row[column])

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
  correct = 0
  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0

########################

###########INPUT
learning_rate=0.01
hid=10
epoche=5
num_rows=1000
####################
# dataset mio quadrato ##############
CSV_url='https://raw.githubusercontent.com/stefanoridolfi/Coding_ML/master/dataset/dataset_quadrato.csv'
#############
df=pd.read_csv(CSV_url,nrows=num_rows)
df = df.values
np.random.seed(5)
np.random.shuffle(df)
train = df[0:int(0.7*len(df))]
test = df[int(0.7*len(df)):int(len(df))]

xy_train = train[:, 0:3]
xy_test = test[:, 0:3]

xy_train_list=xy_train.tolist()
xy_test_list = xy_test.tolist()
float_column_to_int(xy_train_list, 2)
float_column_to_int(xy_test_list, 2)



'''
Creo matrice dei pesi. Input sono 3, x1,x2 e x3 che è BIAS. Il vettore X ha dimensioni 1x3. Lo strato hidden è composto da 4 uscite, y1,y2,y3 e y4 ed è rappresentato dal vettore Y di dimensioni 4x1
Il vettore Y, di dimensioni 4x1 è il risultato del prodotto Wih.dot.X La matrice Wih ha dimensioni 4x3. Y=Wih.dot.X (4x1=4x3.dot.3x1)
L'output Z ha dimensioni 1. La matrice dei pesi da Y a Z,matrice Who,  ha dimensione 1x4. L'output Z ha dimensioni 1x1. Per questo Z=Who.dot.Y (1x1=1x4.dot.4x1)  
'''
#Creo punti Input per training. Per ogni punto aggiungo un campo che vale 1 se il punto è nel quadrato, 0 se è fuori. Nel disegno 1 è punto rosso 0 è punto verde
# defnisco contorni del quadrato


# Creo la matrice Wih, hid x2, i cui elementi sono i pesi. I pesi hanno un valore con distrubuzione gaussiano vm==0 e varianza=1
Wih=np.random.randn(hid,2)

# Creo la matrice Who, 1xhid, i cui elementi sono i pesi. I pesi hanno un valore distrubuzione gaussiano vm==0 e varianza=1
Who=np.random.randn(1,hid)

# per vedere le dimensioni di un array in comando è :array.shape, esempio Wih.shape
# Inizializzo i valori dell'input con una lista. X=(x1,x2).

# Creo il vettore BIAS del livello nascosto il vettore ha dimensioni (hidx1)
b1=[]
for t in range(0,hid):
  b1.append(1)
b1=np.array(b1)
b1=b1.reshape(hid,1)

#Creo il vettore BIAS del livello di uscita, il vettore ha dimensioni (1 x 1)
b2=[]
for t in range(0,1):
  b2.append(1)
b2=np.array(b2)
b2=b2.reshape(1,1)
###############TRAINING################################ 
print("INIZIO TRAINING SET\n")

Epo=[]
Err_y=[]

for e in range(0,epoche):
  Err=0
  Err_rel=0
  for punto in xy_train_list:
    X=(punto[0],punto[1])
# lo devo trasformare in un array altrimenti non posso fare operazioni vettoriali
    X=np.array(X)
    X=X.reshape(2,1)
  

 
#calcolo Y=Wih.dot.X
    Y=np.dot(Wih,X)
#Sommo ad Y il BIAS b1
    Y=Y+b1
 
#calcolo Y_act che è il vettore Y a cui ho applicato la funzione di attivazione
    Y_act=activation_function(Y)
  
# Y ha dimensione 4x1
# Calcolo Z=Who.dot.Y.T
    Z=np.dot(Who,Y_act)
#Sommo ad Z il BIAS b2
    Z=Z+b2
#calcolo Z_act che è il vettore Z a cui ho applicato la funzione di attivazione
    Z_act=activation_function(Z)

 
  #print("Z_act= ", Z_act, "pos_reale=", punto[2])
#calcolo l'errore finale E=valore_corretto - Z_act . Il valore corretto è la terza colonna dei punti di training, ovvero xy_train_list[2]
    error_output=punto[2]-Z_act
    Err=Err+(error_output)**2
    # print("Errore= ",output_error)
#Calcolo il delta da sommare al corrente valore di Y_act: moltiplicazione di learning_rate*output_errore*derivata_sigmoide in Z*valore del vettore Y_act 
# la derivata della sigmoide calcolata nel punto Z_act è: Z_act(1-Z_act)!!
    uno_Z_act=(1-Z_act)
    tmp1=error_output*Z_act*uno_Z_act
    # tmp1 è il delta del livello di uscita
    tmp2=learning_rate*(np.dot(tmp1,Y_act.T))
    # Calcolo d_b2
    d_b2=tmp1
    #d_b2=np.sum(tmp1,axis=1, keepdims=True)
     
  
  # Calcolo matrice Wih
    error_hidden=np.dot(Who.T,tmp1)
    tmp3=error_hidden*Y_act*(1-Y_act)
    #tmp3 è il delta del livello finale
    U=np.dot(tmp3,X.T)
     # Calcolo d_b1
    d_b1=tmp3
    #d_b1=np.sum(tmp3,axis=1, keepdims=True)
  #aggiorno matrice dei pesi Wih
    Wih=Wih+learning_rate*U
   #aggiorno matrice dei pesi Who
    Who=Who+tmp2
   #aggiorno d_b1
    b1=b1+d_b1*learning_rate
   #aggiorno b2
    b2=b2+d_b2*learning_rate
  #print("dim tmp=",tmp.shape,",dim Z_act=", Z_act.shape,",dim (1-Z_act)=",uno_Z_act.shape, ",dim Z_act_deri=",Z_act_deri.shape,",dim Delta_Who=",Delta_Who.shape)
  #print("Z_act=", Z_act,",(1-Z_act)=",uno_Z_act, ",Z_act_deri=",Z_act_deri,",Delta_Who=",Delta_Who)
  #print("Who=", Who)

  Err_rel=(Err)/len(xy_train_list)
  Epo.append(e)
  Err_y.append(Err_rel)
  print("epoca=",e,"Errore =",Err_rel)
  

######################FINE TRAINING#######################################
# TEST SET  
print("INIZIO TEST\n")
predicted=list()
errori_tot=0
for punto in xy_test_list:
   
   X=(punto[0],punto[1])
   X=np.array(X)
   X=X.reshape(2,1)
   Y=np.dot(Wih,X)+b1
   Y_act=activation_function(Y)
   Z=np.dot(Who,Y_act)+b2
   Z_act=activation_function(Z)
# inserisco valore predetto
   predicted.append(posizione_neurale(Z_act)[0])
   if posizione_neurale(Z_act)[0]!=punto[2]:
     #pint("Z_act:formato neurale, output neurale, posizione corretta",Z_act,posizione_neurale(Z_act)[1],",",posizione_neurale(Z_act)[0],posizione(punto,xa,xb,ya,yb)[0])
     errori_tot=errori_tot+1
actual = [row[-1] for row in xy_test_list]
accuracy=accuracy_metric(actual, predicted)
print(commento," punti training=",len(xy_train_list)," epoche=",epoche," punti test=",len(xy_test_list), " learning_rate=",learning_rate, " neuroni strato nascosto=",hid)
print("errori totali=",errori_tot, "su", len(xy_test_list), "%",(errori_tot/len(xy_test_list))*100)
print("accuracy native python: %2.2f" % accuracy)

#################################USING SCIKIT###################
from sklearn.neural_network import MLPClassifier
CSV_url='https://raw.githubusercontent.com/stefanoridolfi/Coding_ML/master/dataset/dataset_quadrato_with_bias.csv'
df1=pd.read_csv(CSV_url,nrows=num_rows)
df1=df1.values
np.random.seed(5)
np.random.shuffle(df1)


train1 = df1[0:int(0.7*len(df1))]
test1 = df1[int(0.7*len(df1)):int(len(df1))]



x_train1 = train1[:,0:3] 
y_train1 = train1[:, 3] 

x_test1 = test1[:, 0:3]
y_test1 = test1[:, 3]
clf = MLPClassifier(solver='lbfgs', activation='relu',alpha=1e-5,hidden_layer_sizes=(10), random_state=1)
clf.fit(x_train1, y_train1)
predicted1=clf.predict(x_test1)
accuracy1=accuracy_metric(y_test1, predicted1)
print("accuracy scikit: %2.2f" % accuracy1)



INIZIO TRAINING SET

epoca= 0 Errore = [[0.33834771]]
epoca= 1 Errore = [[0.08327731]]
epoca= 2 Errore = [[0.05234813]]
epoca= 3 Errore = [[0.04623417]]
epoca= 4 Errore = [[0.04384528]]
INIZIO TEST

VERSIONE con BIAS  punti training= 700  epoche= 5  punti test= 300  learning_rate= 0.01  neuroni strato nascosto= 10
errori totali= 15 su 300 % 5.0
accuracy native python: 95.00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in power


accuracy scikit: 95.00
